In [14]:
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp
import matplotlib.pyplot as plt

In [15]:
# problem 1 settings
n = 2
mu = matrix([[0.05],[0.10]])
sig = matrix([[0.01,-0.01],[-0.01,0.04]])
q = matrix(0.0,(n,1))
A = matrix(1.0,(1,n))
b = matrix(1.0)
min_var_port = qp(sig,q,A=A,b=b)['x']
print(min_var_port)

[ 7.14e-01]
[ 2.86e-01]

